# Ejercicio: Crear un DataFrame Completo con SWAPI

## Objetivo:
El objetivo de este ejercicio es que los alumnos practiquen la obtención de datos de una API REST y su posterior procesamiento para crear un DataFrame en Pandas que contenga información detallada sobre personajes de Star Wars.

## Instrucciones:

1. Obtener los datos:

- Utilizar la API de Star Wars (SWAPI) para obtener datos sobre los personajes (people) de la saga.
- La URL base para obtener la lista de personajes es: https://swapi.dev/api/people/.
- Dado que los datos están paginados, deberán recorrer todas las páginas para obtener la lista completa de personajes.

2. Crear el DataFrame:

    Crear un DataFrame que incluya las siguientes columnas con la información detallada de cada personaje:

    - id: Identificador único del personaje.
    - name: Nombre del personaje.
    - height: Altura del personaje (en centímetros).
    - mass: Peso del personaje (en kilogramos).
    - hair_color: Color de cabello.
    - skin_color: Color de piel.
    - eye_color: Color de ojos.
    - birth_year: Año de nacimiento.
    - gender: Género.
    - homeworld: Nombre del planeta de origen (obtenido mediante una petición adicional a la API).
    - species: Nombre de la especie del personaje (obtenido mediante una petición adicional si el campo species no está vacío).
    - films: Lista de nombres de películas en las que aparece el personaje (obtenido mediante peticiones adicionales).

3. Procesamiento de datos:

    - Cada entrada en el campo homeworld de los personajes es un enlace a la API, desde el cual se debe obtener el nombre del planeta.

    - El campo species contiene una lista de enlaces. Si no está vacío, se debe obtener el nombre de la especie desde la API.

    - El campo films contiene una lista de enlaces a las películas. Se debe realizar una petición por cada enlace para obtener el título de la película.

4. Entrega del ejercicio:

    - Se debe entregar un archivo .ipynb (Jupyter Notebook) con el código que obtenga los datos, procese las respuestas y construya el DataFrame completo.

    - También deben incluir en el notebook una breve descripción de cómo organizaron el proceso para obtener los datos y construir el DataFrame.

Tips:

- Utilicen la librería requests para realizar las peticiones HTTP.
- Tengan en cuenta que la API utiliza paginación, por lo que deberán iterar sobre las páginas hasta obtener todos los datos.
- Los datos relacionados (homeworld, species, films) requieren realizar peticiones adicionales, así que gestionen adecuadamente los tiempos de espera (pueden considerar utilizar la función time.sleep() para evitar sobrecargar la API).
- Si encuentran campos con datos faltantes o errores, documenten cómo lo manejan en su notebook.

In [5]:
import hashlib
import requests
import datetime
import pandas as pd

In [23]:
def get_all_data(base_url):
    all_data = []
    current_url = base_url

    while current_url:
        response = requests.get(current_url)
        data = response.json()

        all_data.extend(data["results"])
        current_url = data["next"]

    return all_data

uri_charachters = "https://swapi.dev/api/people/"
charachters = get_all_data(uri_charachters)


uri_planets = "https://swapi.dev/api/planets/"
planets = get_all_data(uri_planets)

uri_species = "https://swapi.dev/api/species/"
species = get_all_data(uri_species)


uri_films = "https://swapi.dev/api/films/"
films = get_all_data(uri_films)


def handle_data(charachters, planets, species, films):
    data_list = []

    planet_name = {planet["url"]: planet["name"] for planet in planets}
    specie_name = {specie["url"]: specie["name"] for specie in species}
    film_name = {film["url"]: film["title"] for film in films}
    
    for char in charachters:
       
        char_species = char["species"]
        given_specie = [specie_name.get(specie_url, "Unknown") for specie_url in char_species]
        given_specie = ", ".join(given_specie) if given_specie else "Unknown"

        char_films = char["films"]
        given_film = [film_name.get(film_url, "Unknown") for film_url in char_films]

        data_list.append(
            {
                "id": char["url"].split("/")[-2],
                "name": char["name"],
                "height": char["height"],
                "mass": char["mass"],
                "hair_color": char["hair_color"],
                "skin_color": char["skin_color"],
                "eye_color": char["eye_color"],
                "birth_year": char["birth_year"],
                "gender": char["gender"],
                "homeworld": planet_name.get(char["homeworld"], "Unknown"),
                "species": given_specie,
                "films": given_film
            }
        )

    return data_list

handle_data(charachters, planets, species, films)

[{'id': '1',
  'name': 'Luke Skywalker',
  'height': '172',
  'mass': '77',
  'hair_color': 'blond',
  'skin_color': 'fair',
  'eye_color': 'blue',
  'birth_year': '19BBY',
  'gender': 'male',
  'homeworld': 'Tatooine',
  'species': 'Unknown',
  'films': ['A New Hope',
   'The Empire Strikes Back',
   'Return of the Jedi',
   'Revenge of the Sith']},
 {'id': '2',
  'name': 'C-3PO',
  'height': '167',
  'mass': '75',
  'hair_color': 'n/a',
  'skin_color': 'gold',
  'eye_color': 'yellow',
  'birth_year': '112BBY',
  'gender': 'n/a',
  'homeworld': 'Tatooine',
  'species': 'Droid',
  'films': ['A New Hope',
   'The Empire Strikes Back',
   'Return of the Jedi',
   'The Phantom Menace',
   'Attack of the Clones',
   'Revenge of the Sith']},
 {'id': '3',
  'name': 'R2-D2',
  'height': '96',
  'mass': '32',
  'hair_color': 'n/a',
  'skin_color': 'white, blue',
  'eye_color': 'red',
  'birth_year': '33BBY',
  'gender': 'n/a',
  'homeworld': 'Naboo',
  'species': 'Droid',
  'films': ['A New Ho